In [77]:
import pandas as pd
import plotly.express as px
from datetime import datetime
from collections import Counter
import re

## Acronyms & nomenclature
* `Rec`: Réception (ou enregistrement).
* `Inv`: Investigation (ou examen).

## Sample size N = 110

## 1. Read initial file sent to me

In [78]:
df = pd.read_excel('Everything merged 18.12._new.xlsx')
print(df.shape)
df.head()

(110, 71)


,PatientID,DateRecInvIRM,ResultatIRMRecInv,PIRADSLesionRecInv,ResultatBiopsieRecInv,VolProstateIRM,PSA,Column8,2nd mri date,2nd mri resultat,...,all patients.age_capra,all patients.D28VolD90,all patients.D28VolV100,pet.PatientID,pet.DateRecInvRadiative,pet.TypeRecInvRadiative,pet.ResultatRecInvRadiative,date treatment.PatientId,date treatment.TypeTX,date treatment.DateBrachy
0,5753253,2024-05-18,Positive,4.0,Positive,27.7,4.10,NaN,NaT,NaN,...,1,140.9,89.3,5753253.0,2024-09-12,PET ... type,Positif,5753253,Curietherapie LDR,2022-02-24
1,5725736,2024-01-13,Négative,NaN,NaN,NaN,6.00,NaN,NaT,NaN,...,1,141.3,89.6,NaN,NaT,NaN,NaN,5725736,Curietherapie LDR,2022-01-04
2,808919,2024-02-04,Négative,NaN,NaN,30.4,5.87,NaN,NaT,NaN,...,1,164.2,95.4,NaN,NaT,NaN,NaN,808919,Curietherapie LDR,2021-05-27
3,5687019,2023-11-18,Négative,3.0,Négative,18.0,4.20,NaN,NaT,NaN,...,1,157.0,94.2,NaN,NaT,NaN,NaN,5687019,Curietherapie LDR,2021-03-25
4,5339331,2023-06-24,Positive,NaN,NaN,28.0,3.78,NaN,NaT,NaN,...,1,159.0,96.0,5339331.0,2023-08-03,PET ... type,Positif,5339331,Curietherapie LDR,2021-03-11


## 2. Remove empty columns
9 empty columns removed: `['Column8', 'Column12', 'Column15', 'Column18', 'Column19', '4th mri date', 'Column25', 'RecInvbiopsie.PSA', 'all patients.causedeces']`

In [79]:
empty_cols = df.columns[df.isna().all()].tolist()
print(empty_cols)

['Column8', 'Column12', 'Column15', 'Column18', 'Column19', '4th mri date', 'Column25', 'RecInvbiopsie.PSA', 'all patients.causedeces']


In [80]:
df = df.drop(columns=empty_cols)

## 3. Remove duplicate columns
3 duplicate columns removed: `['date treatment.TypeTX', 'date treatment.PatientId', 'all patients.PatientId']`

In [81]:
duplicate_cols = df.T.duplicated(keep=False)  # `keep=False` marks all duplicates
dup_col_names = df.columns[duplicate_cols].tolist() # Get the names of duplicate columns

for i,dup in enumerate(dup_col_names):
    for j in range(i+1,len(dup_col_names)):
        if df[dup].equals(df[dup_col_names[j]]): print(f"{dup} EQUALS {dup_col_names[j]}")

PatientID EQUALS all patients.PatientId
PatientID EQUALS date treatment.PatientId
all patients.PatientId EQUALS date treatment.PatientId
all patients.TypeTX EQUALS date treatment.TypeTX


In [82]:
print(len(df['RecInvbiopsie.PatientID'].notna() & (df['RecInvbiopsie.PatientID'] == df['PatientID'])))

110


In [83]:
print(len(df['RecInvbiopsie.PatientID'].notna() & (df['pet.PatientID'] == df['PatientID'])))

110


In [84]:
remove_dup_cols = ['date treatment.TypeTX', 'date treatment.PatientId', 
                   'all patients.PatientId','RecInvbiopsie.PatientID','pet.PatientID']
df = df.drop(columns=remove_dup_cols)

## 4. Rename & Re-Order Columns

_More automated way to rename the columns (but I am not using this currently)._

In [ ]:
# col_list = list(df.columns)
# df = df.rename(columns=lambda x: x.replace("all patients.", ""))
# col_list = list(df.columns)
# col_list = [re.sub(r'([a-z])([A-Z])', r'\1_\2', col).lower() for col in col_list]
# col_list = [s.rstrip() for s in col_list]
# col_list = [s.replace("  ", " ") for s in col_list]
# col_list = [s.replace(" ", "_") for s in col_list]
# col_list = [s.replace("aps", "psa") for s in col_list]
# col_list = [s.replace("treatment", "tx") for s in col_list]
# col_list = [s.replace("biopsie", "biopsy") for s in col_list]
# col_list = [s.replace("resultat", "result") for s in col_list]

# # Rename dataframe columns
# df.columns = col_list

In [87]:
df = df.rename(columns={
    'PatientID':'patient_id',
    'all patients.Age':'tx-age',
    'all patients.ÉchantPélevés':'tx-biopsy_num_samples',
    'all patients.ÉchantPositifs':'tx-biopsy_num_positive',
    'all patients.GleasonTotal':'tx-gleason_total',
    'all patients.T':'tx-t_stage',
    'all patients.protocole':'tx-protocol',
    'date treatment.DateBrachy':'tx-date',
    'all patients.TypeTX':'tx-type',
    'all patients.DoseTotalProstate':'tx-total_dose_prostate',
    'all patients.D28VolD90': 'tx-d28_vol_d90',
    'all patients.D28VolV100':'tx-d28_vol_v100',
    'all patients.ADT':'tx-adt',
    'all patients.ApsMonth':'psa-time_since_tx',
    'all patients.Aps':'psa-val',
    'all patients.nadiraps02':'psa-nadir_02',
    'all patients.nadiraps05':'psa-nadir_05',
    'all patients.CAPRA':'psa-capra_total',
    'all patients.aps_capra':'psa-capra_psa',
    'all patients.gleason_capra':'psa-capra_gleason',
    'all patients.tstage_capra':'psa-capra_t_stage',
    'all patients.biopsy_capra':'psa-capra_biopsy',
    'all patients.age_capra':'psa-capra_age',
    'all patients.ddeces':'date_death',
    'all patients.biochemical recurrence':'bf-date',
    'DateRecInvIRM':'mri_1-date',
    'ResultatIRMRecInv':'mri_1-result',
    'PIRADSLesionRecInv':'mri_1-pirads_score',
    'VolProstateIRM':'mri_1-prostate_vol',
    'PSA':'mri_1-psa',
    '2nd_MRI_Date':'mri_2-date',
    '2nd_MRI_Result':'mri_2-result',
    '2nd_MRI_Vol':'mri_2-prostate_vol',
    '2nd_MRI_PSA':'mri_2-psa',
    '3rd_MRI_Date':'mri_3-date',
    '3rd_MRI_Result':'mri_3-result',
    '3rd_MRI_Vol':'mri_3-prostate_vol',
    '3rd_MRI_PSA':'mri_3-psa',
    '4th_MRI_Date':'mri_4-date',
    '4th_MRI_Result':'mri_4-result',
    '4th_MRI_Vol':'mri_4-prostate_vol',
    '4th_MRI_PSA':'mri_4-psa',
    'RecInvbiopsie.DateRecInvBiopsie':'biopsy-date',
    'RecInvbiopsie.ResultatRecInvBiopsie':'biopsy-result',
    'RecInvbiopsie.GleasonPrimRecInv':'biopsy-gleason_1ary',
    'RecInvbiopsie.GleasonSecRecInv':'biopsy-gleason_2ary',
    'RecInvbiopsie.GleasonTertRecInv':'biopsy-gleason_3ary',
    'pet.DateRecInvRadiative':'pet-date',
    'pet.ResultatRecInvRadiative':'pet-result'
})

# To finish addressing:

### Quick look at initially cleaned dataset + metadata

### Preproc: Duplicate Patients
* `2060194`: duplicates are NOT the same; later biopsy (RecInvbiopsie.ResultatRecInvBiopsie; 2015) was positive.
* `2054846`: duplicates are NOT the same; later biopsy (RecInvbiopsie.ResultatRecInvBiopsie; 2021) was positive.
* `2205490`: duplicates are NOT the same; biopsy (RecOmvbiopsie.Column8)in 2018 was positive

### Preproc: save file with date and time in SPECIFIC TIME ZONE

### Preproc:  add column of ratio b/w positive samples and samples taken!

### "Converters" Patients who initially had a +ve MRI that became negative
* `7049838`: LDR, +ve 1st MRI, +ve 2nd MRI, ? 3rd MRI, -ve 4th MRI, –ve recurrence biopsy.
* `7051280`: HDR, +ve 1st MRI, +ve 2nd MRI, –ve 3rd MRI, –ve recurrence biopsy.
* `5519879`: LDR, +ve 1st MRI, –ve 2nd MRI, ??? recurrence biopsy.

STRATEGY: try to include, then exclude. Add column called "converter". Mention in discussion.

### Analysis:
- Stratify by Tx type?
- Positive recurrence: keep data of initial positive MRI and initial positive biopsy (plus time between!!!).
- Add data from downstream positives as extra patient (but mention it is a duplicate).
- Negative (no recurrence): keep data of latest negative.

### Weird patients
Note that by MRI #2, these patient's PSA levels almost doubled!
* `2102593`: +ve 1st MRI, +ve 2nd MRI PSA LEVELS DOUBLE, +ve biopsy
* `2205490`: +ve 1st MRI, +ve 2nd MRI, –ve biopsy
* `2162006`: +ve 1st MRI, +ve 2nd MRI, –ve biopsy